In [68]:
import numpy as np
import os
import cv2
from tqdm import tqdm

In [69]:
# Paths
ORIGINAL_IMAGES_DIR = '/home/KutumLabGPU/Documents/santosh/TNBC-project/input-dir/pannuke/patches' 
ALL_MASKS_NPY_PATH = '/media/network/hdd/public_datasets_archive/01_notebooks/pannuke/raw_files4inference/masks/masks.npy'
OUTPUT_OVERLAY_DIR = '/home/KutumLabGPU/Documents/santosh/TNBC-project/input-dir/pannuke/overlay/overlays-legend' 
if not os.path.exists(OUTPUT_OVERLAY_DIR):
    os.makedirs(OUTPUT_OVERLAY_DIR)

In [70]:
class_info = {
    0: {'name': 'Neoplastic', 'color': (0, 0, 255)}, # Red
    1: {'name': 'Inflammatory', 'color': (0, 255, 0)}, # Green
    2: {'name': 'Connective', 'color': (255, 0, 0)}, # Blue
    3: {'name': 'Dead', 'color': (0, 255, 255)},     # Yellow
    4: {'name': 'Epithelial', 'color': (255, 255, 0)},  # Cyan
}

In [71]:
legend_items = [(info['name'], info['color']) for idx, info in sorted(class_info.items())]

In [72]:
print(f"Loading masks from {ALL_MASKS_NPY_PATH}...")

all_masks = np.load(ALL_MASKS_NPY_PATH, allow_pickle=True) 
print(f"Masks loaded successfully. Shape: {all_masks.shape}")

if all_masks.dtype == 'object':
    print("Converting object array to standard numpy array...")
    all_masks = np.stack(all_masks, axis=0)
    print(f"Conversion successful. New shape: {all_masks.shape}")

Loading masks from /media/network/hdd/public_datasets_archive/01_notebooks/pannuke/raw_files4inference/masks/masks.npy...
Masks loaded successfully. Shape: (10, 256, 256, 6)
Converting object array to standard numpy array...
Conversion successful. New shape: (10, 256, 256, 6)


In [73]:
print(f"Listing image files from {ORIGINAL_IMAGES_DIR}...")
image_files = sorted([f for f in os.listdir(ORIGINAL_IMAGES_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff'))])
print(f"Found {len(image_files)} image files.")

Listing image files from /home/KutumLabGPU/Documents/santosh/TNBC-project/input-dir/pannuke/patches...
Found 10 image files.


In [74]:
first_image_path = os.path.join(ORIGINAL_IMAGES_DIR, image_files[0])
first_img = cv2.imread(first_image_path)
if first_img is None:
    print(f"Error loading first image {first_image_path}. Check file integrity.")
    exit()
img_height, img_width = first_img.shape[:2]

mask_num_masks, mask_height, mask_width, num_mask_channels = all_masks.shape
expected_mask_channels = len(class_info) + 1

print(f"Image dimensions: ({img_width}x{img_height})")
print(f"Mask dimensions: ({mask_width}x{mask_height})")

Image dimensions: (1024x1024)
Mask dimensions: (256x256)


In [75]:
scale_factor_x = img_width / mask_width
scale_factor_y = img_height / mask_height

if scale_factor_x != 1.0 or scale_factor_y != 1.0:
    print(f"Mask dimensions are smaller than image dimensions. Scaling contours by ({scale_factor_x}, {scale_factor_y}).")
else:
    print("Mask and image dimensions match. No scaling needed.")


Mask dimensions are smaller than image dimensions. Scaling contours by (4.0, 4.0).


In [76]:
if num_mask_channels != expected_mask_channels:
     print(f"Warning: Mask .npy has {num_mask_channels} channels ({all_masks.shape[-1]}), but expected {expected_mask_channels} based on class_info (+ background).")
     print("Proceeding assuming channels 0-4 correspond to the classes for overlay.")
     if all_masks.shape[-1] < len(class_info):
          print(f"Error: Mask only has {all_masks.shape[-1]} channels, fewer than the {len(class_info)} classes defined for overlay. Exiting.")
          exit()

In [77]:
# Legend parameters

legend_start_x_offset_from_right = 250 # Start 250 pixels from the right edge
legend_start_y = 20 # Start 20 pixels from the top
color_box_size = 20
text_offset_x = color_box_size + 10 
line_height = 30 
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.7
font_thickness = 2 
text_color = (0, 0, 0) 
legend_bg_padding = 10 
alpha_value = 0.5 

# Pre-calculate legend background dimensions and position 

legend_start_x = img_width - legend_start_x_offset_from_right 

legend_bg_width = 230 
legend_bg_height = len(legend_items) * line_height + 2 * legend_bg_padding

legend_bg_rect_start = (max(0, legend_start_x - legend_bg_padding), max(0, legend_start_y - legend_bg_padding))
legend_bg_rect_end = (min(img_width, legend_start_x - legend_bg_padding + legend_bg_width), min(img_height, legend_start_y - legend_bg_padding + legend_bg_height))

In [78]:
print("Starting overlay process...")

for i, (image_file, current_mask) in tqdm(enumerate(zip(image_files, all_masks)), total=len(image_files), desc="Processing Images"):
    image_path = os.path.join(ORIGINAL_IMAGES_DIR, image_file)
    
    image = cv2.imread(image_path) 
    
    if image is None:
        tqdm.write(f"Warning: Could not load image {image_file}. Skipping.")
        continue
        
    if image.shape[:2] != (img_height, img_width):
         tqdm.write(f"Warning: Image {image_file} has unexpected dimensions {image.shape[:2]}. Skipping.")
         continue

    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    final_image_with_overlay = image.copy() 
    
    for class_idx in range(len(class_info)): 
        
        if class_idx >= current_mask.shape[-1]:
             continue

        class_mask = current_mask[:, :, class_idx]
        instance_ids = np.unique(class_mask[class_mask > 0])

        if len(instance_ids) > 0:
            color = class_info[class_idx]['color']
            
            for instance_id in instance_ids:
                instance_binary_mask = (class_mask == instance_id).astype(np.uint8) * 255

                contours, _ = cv2.findContours(instance_binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                scaled_contours = []
                for contour in contours:
                    if contour.shape[0] == 0:
                         continue

                    scaled_points = contour.squeeze().astype(np.float32) 
                    
                    if scaled_points.ndim == 1:
                         scaled_points = scaled_points[np.newaxis, :]

                    scaled_points[:, 0] *= scale_factor_x
                    scaled_points[:, 1] *= scale_factor_y

                    scaled_points = np.round(scaled_points).astype(np.int32)
                    scaled_contour = scaled_points[:, np.newaxis, :]
                    scaled_contours.append(scaled_contour)

                # Draw the boundary with thickness 2 pixels
                cv2.drawContours(final_image_with_overlay, scaled_contours, -1, color, 2) 

    # Draw the Legend
    bg_x1, bg_y1 = legend_bg_rect_start
    bg_x2, bg_y2 = legend_bg_rect_end
    
    if bg_x2 > bg_x1 and bg_y2 > bg_y1:
        sub_img = final_image_with_overlay[bg_y1:bg_y2, bg_x1:bg_x2]
        white_rect = np.full(sub_img.shape, 255, dtype=np.uint8)
        cv2.addWeighted(sub_img, 1 - alpha_value, white_rect, alpha_value, 0, sub_img)
        final_image_with_overlay[bg_y1:bg_y2, bg_x1:bg_x2] = sub_img

    current_y = legend_start_y
    for name, color_bgr in legend_items:
        cv2.rectangle(
            final_image_with_overlay,
            (legend_start_x, current_y),
            (legend_start_x + color_box_size, current_y + color_box_size),
            color_bgr,
            -1 
        )
        
        text_x = legend_start_x + text_offset_x
        text_y = current_y + color_box_size - 5 

        if text_x < img_width and text_y < img_height:
             cv2.putText(
                 final_image_with_overlay,
                 name,
                 (text_x, text_y),
                 font,
                 font_scale,
                 text_color,
                 font_thickness,
                 cv2.LINE_AA
             )
             
        current_y += line_height 

    output_filename = f"gt-overlaid-legend_{os.path.splitext(image_file)[0]}.png"
    output_path = os.path.join(OUTPUT_OVERLAY_DIR, output_filename)

    success = cv2.imwrite(output_path, final_image_with_overlay)
    if not success:
         tqdm.write(f"Warning: Could not save overlaid image {output_path}.")

print("Overlay process finished.")
print(f"Overlaid images saved to {OUTPUT_OVERLAY_DIR}")

Starting overlay process...


Processing Images: 100%|██████████| 10/10 [00:00<00:00, 10.29it/s]

Overlay process finished.
Overlaid images saved to /home/KutumLabGPU/Documents/santosh/TNBC-project/input-dir/pannuke/overlay/overlays-legend


In [79]:
# Remove the legends
print("Starting overlay process without legend...")

OUTPUT_OVERLAY_DIR_N0_LEGEND = '/home/KutumLabGPU/Documents/santosh/TNBC-project/input-dir/pannuke/overlay/overlays-no-legend' 
if not os.path.exists(OUTPUT_OVERLAY_DIR_N0_LEGEND):
    os.makedirs(OUTPUT_OVERLAY_DIR_N0_LEGEND)

print(f"Directory created at {OUTPUT_OVERLAY_DIR_N0_LEGEND}")

Starting overlay process without legend...
Directory created at /home/KutumLabGPU/Documents/santosh/TNBC-project/input-dir/pannuke/overlay/overlays-no-legend


In [80]:
for i, (image_file, current_mask) in tqdm(enumerate(zip(image_files, all_masks)), total=len(image_files), desc="Processing Images"):
    image_path = os.path.join(ORIGINAL_IMAGES_DIR, image_file)
    
    image = cv2.imread(image_path) 
    
    if image is None:
        tqdm.write(f"Warning: Could not load image {image_file}. Skipping.")
        continue
        
    if image.shape[:2] != (img_height, img_width):
         tqdm.write(f"Warning: Image {image_file} has unexpected dimensions {image.shape[:2]}. Skipping.")
         continue

    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    final_image_with_overlay = image.copy() 
    
    for class_idx in range(len(class_info)): 
        
        if class_idx >= current_mask.shape[-1]:
             continue

        class_mask = current_mask[:, :, class_idx]
        instance_ids = np.unique(class_mask[class_mask > 0])

        if len(instance_ids) > 0:
            color = class_info[class_idx]['color']
            
            for instance_id in instance_ids:
                instance_binary_mask = (class_mask == instance_id).astype(np.uint8) * 255

                contours, _ = cv2.findContours(instance_binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                scaled_contours = []
                for contour in contours:
                    if contour.shape[0] == 0:
                         continue

                    scaled_points = contour.squeeze().astype(np.float32) 
                    
                    if scaled_points.ndim == 1:
                         scaled_points = scaled_points[np.newaxis, :]

                    scaled_points[:, 0] *= scale_factor_x
                    scaled_points[:, 1] *= scale_factor_y

                    scaled_points = np.round(scaled_points).astype(np.int32)
                    scaled_contour = scaled_points[:, np.newaxis, :]
                    scaled_contours.append(scaled_contour)

                cv2.drawContours(final_image_with_overlay, scaled_contours, -1, color, 2) # Draw boundary, thickness 2

    output_filename = f"gt-overlaid_{os.path.splitext(image_file)[0]}.png"
    output_path = os.path.join(OUTPUT_OVERLAY_DIR_N0_LEGEND, output_filename) 

    success = cv2.imwrite(output_path, final_image_with_overlay)
    if not success:
         tqdm.write(f"Warning: Could not save overlaid image {output_path}.")

print("Overlay process finished.")
print(f"Overlaid images saved to {OUTPUT_OVERLAY_DIR_N0_LEGEND}")

Processing Images: 100%|██████████| 10/10 [00:00<00:00, 10.93it/s]

Overlay process finished.
Overlaid images saved to /home/KutumLabGPU/Documents/santosh/TNBC-project/input-dir/pannuke/overlay/overlays-no-legend
